In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: main-workspace
Azure region: eastus2
Subscription id: 167b9ae9-f575-4021-810b-253a2e5be2bd
Resource group: mlops


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "kamau100"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    cluster = ComputeTarget(
        workspace=ws,
        name = cluster_name
    )
    print('Cluster in existence')
except:
    config = AmlCompute.provisioning_configuration(
    vm_size= 'STANDARD_V2_V2',
    max_nodes = 4)
    cluster = ComputeTarget.create(
        ws,
        cluster_name,
        config)



cluster.wait_for_completion(show_output=True)

Cluster in existence
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": choice(0.01, 5, 20, 100, 500),
    "--max_iter": choice(10,50,100,150,200)
})

# Specify a Policy
policy = BanditPolicy(
    slack_factor = 0.1,
    evaluation_interval= 2,
    delay_evaluation= 10,
)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory= "./",
    script = "train.py",
    compute_target = cluster,
    environment=sklearn_env
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling = ps,
    primary_metric_name= 'Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=4

)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(
    config = hyperdrive_config,
    show_output = True
)
hyperdrive_run.wait_for_completion(show_output=True)
RunDetails(hyperdrive_run).show()

RunId: HD_48740302-4259-4401-a7c0-084b6e11a26b
Web View: https://ml.azure.com/runs/HD_48740302-4259-4401-a7c0-084b6e11a26b?wsid=/subscriptions/167b9ae9-f575-4021-810b-253a2e5be2bd/resourcegroups/mlops/workspaces/main-workspace&tid=292035c0-a3ff-49d4-9b39-8cab5c9e0b53

Streaming azureml-logs/hyperdrive.txt

[2022-07-20T09:03:51.348285][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-07-20T09:03:52.2867837Z][SCHEDULER][INFO]Scheduling job, id='HD_48740302-4259-4401-a7c0-084b6e11a26b_2' 
[2022-07-20T09:03:52.0284599Z][SCHEDULER][INFO]Scheduling job, id='HD_48740302-4259-4401-a7c0-084b6e11a26b_0' 
[2022-07-20T09:03:52.369152][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2022-07-20T09:03:52.4196073Z][SCHEDULER][INFO]Scheduling job, id='HD_48740302-4259-4401-a7c0-084b6e11a26b_3' 
[2022-07-20T09:03:52.1623798Z][SCHEDULER][INFO]Scheduling job, id='HD_48740302-4259-4401-a7c0-084b6e11a26b_1' 
[2022-07-20T09:03:

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [6]:
print(hyperdrive_run.get_status())

Completed


In [7]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best run:', best_run.id)
print('Accuracy:', best_run_metrics['Accuracy'])

Best run: HD_48740302-4259-4401-a7c0-084b6e11a26b_11
Accuracy: 0.915270696771061


In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

dataset = TabularDatasetFactory.from_delimited_files(url)

In [9]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(dataset)

In [10]:
import pandas as pd
df_clean = pd.concat([x,y], axis=1)

# Split the data
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df_clean, test_size=0.2, random_state = 42)

# upload data to datastore
from azureml.core import Dataset
os.makedirs('data', exist_ok=True)
train_path = 'data/train.csv'
df_train.to_csv(train_path)

datastore = ws.get_default_datastore()
datastore.upload(
    src_dir = 'data',
    target_path = 'data'
)
ds = Dataset.Tabular.from_delimited_files(datastore.path('data/train.csv'))

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


In [11]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=3)

In [12]:
# Submit your automl run

automl_run = exp.submit(
    config = automl_config,
    show_output = True
)
automl_run.wait_for_completion(show_output=True)
RunDetails(automl_run).show()

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_10950197-d3c5-4f83-8f39-e137a48bef8d,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one cl

2022-07-20:09:31:43,272 WARNING  [connectionpool.py:810] Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))': /jasmine/v1.0/subscriptions/167b9ae9-f575-4021-810b-253a2e5be2bd/resourceGroups/mlops/providers/Microsoft.MachineLearningServices/workspaces/main-workspace/experimentids/5be7459d-2f90-4d1f-8480-e29fd4d2df1b/runs/AutoML_10950197-d3c5-4f83-8f39-e137a48bef8d/next?startChildRun=false
2022-07-20:09:59:44,994 WARNING  [connectionpool.py:810] Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))': /history/v1.0/subscriptions/167b9ae9-f575-4021-810b-253a2e5be2bd/resourceGroups/mlops/providers/Microsoft.MachineLearningServices/workspaces/main-workspace/experimentids/5be7459d-2f90-4d1f-8480-e29fd4d

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_10950197-d3c5-4f83-8f39-e137a48bef8d,automl,Running,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|2951                          |1                               |26360                                 |
+------------------------------+--------------------------------+----

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [13]:
# Retrieve and save your best automl model.

best_run, fitted_model = automl_run.get_output()
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_10950197-d3c5-4f83-8f39-e137a48bef8d_39,,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [14]:
best_run_metrics = best_run.get_metrics()
best_run_metrics

{'balanced_accuracy': 0.7200044554775579,
 'recall_score_weighted': 0.9168816616349265,
 'average_precision_score_weighted': 0.9552617735614262,
 'f1_score_macro': 0.7553432459167754,
 'norm_macro_recall': 0.44000891095511596,
 'AUC_weighted': 0.9472372591007204,
 'average_precision_score_micro': 0.9811820187161852,
 'f1_score_weighted': 0.9096338491991585,
 'weighted_accuracy': 0.9657449890065483,
 'precision_score_macro': 0.813215848469099,
 'f1_score_micro': 0.9168816616349265,
 'recall_score_macro': 0.7200044554775579,
 'recall_score_micro': 0.9168816616349265,
 'log_loss': 0.20475900829848862,
 'precision_score_weighted': 0.9080443137079284,
 'AUC_macro': 0.9472372591007204,
 'precision_score_micro': 0.9168816616349265,
 'average_precision_score_macro': 0.8246496992666823,
 'accuracy': 0.9168816616349265,
 'matthews_correlation': 0.5248779384844862,
 'AUC_micro': 0.980545057412583,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_10950197-d3c5-4f83-8f39-e137a48bef8

In [15]:
fitted_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('35', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bylevel=0.6, colsample_bytree=1, eta=0.001, gamma=0, max_depth=6, max_leaves=15, n_estimators=800, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0, reg_lambda=2.5, subsample=1, tree_method='auto'))], verbose=False))

In [16]:
# We will now register the model
best_run.register_model(
    model_path = './outputs/',
    model_name = 'best_automl_model.pkl',
    properties={'AUC': best_run_metrics['AUC_weighted'],
                'Accuracy': best_run_metrics['accuracy']}
)

Model(workspace=Workspace.create(name='main-workspace', subscription_id='167b9ae9-f575-4021-810b-253a2e5be2bd', resource_group='mlops'), name=best_automl_model.pkl, id=best_automl_model.pkl:1, version=1, tags={}, properties={'AUC': '0.9472372591007204', 'Accuracy': '0.9168816616349265'})

In [ ]:
# Delete the compute
# cluster.delete()

# cluster